In [3]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days, date_add_times
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature
from func.parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel
import gensim
from gensim import corpora

In [17]:
# train_texts = read_pkl_gzip('../feature/eda_base/129__combi92_cnt_card-addr-domain-C-V-M_train.gz')
# test_texts = read_pkl_gzip('../feature/eda_base/129__combi92_cnt_card-addr-domain-C-V-M_test.gz')

def word_split(texts):
    list_text = [text.split(' ') for text in texts]
    return list_text

# train_texts = word_split(train_texts)
# test_texts = word_split(test_texts)
all_texts = train_texts + test_texts

In [25]:
# 単語の出現回数を格納するfrequency変数を定義
from collections import defaultdict

def check_word_freq(list_text):
    frequency = defaultdict(int)
    
    # 単語の出現回数をfrequency変数でカウント
    for text in list_text:
        for token in text:
            frequency[token] += 1
    
    # frequency変数で1より上の単語のみを配列に構築
    texts = [[token for token in text if frequency[token] > 1] for text in list_text]
    return texts
    
texts = check_word_freq(all_texts)

In [26]:
dictionary = corpora.Dictionary(texts)
dictionary.save('../output/09151600_ieee_LDA_gensim.dict')
corpus = [dictionary.doc2bow(text) for text in texts]

In [27]:
# for num_topics in range(5, 13, 2):
num_topics = 5
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)

In [53]:
lda.show_topics()

[(0,
  '0.011*"f__cnt_V283_36001" + 0.011*"f__cnt_V307_1" + 0.010*"f__cnt_V62_38572" + 0.010*"f__cnt_V294_52137" + 0.009*"f__cnt_V281_31295" + 0.008*"f__cnt_V294_24611" + 0.008*"f__cnt_C12_185008" + 0.008*"f__cnt_V67_25167" + 0.008*"f__cnt_V282_22981" + 0.008*"f__cnt_V283_17209"'),
 (1,
  '0.014*"f__cnt_V78_903415" + 0.014*"f__cnt_C7_961240" + 0.014*"f__cnt_addr2_956415" + 0.013*"f__cnt_V94_851784" + 0.013*"f__cnt_card3_956845" + 0.013*"f__cnt_V318_998337" + 0.013*"f__cnt_V317_972197" + 0.013*"f__cnt_V294_972197" + 0.013*"f__cnt_V306_910878" + 0.012*"f__cnt_V38_768706"'),
 (2,
  '0.096*"f__cnt_C3_14643" + 0.034*"f__cnt_C6_10838" + 0.033*"f__cnt_C14_10797" + 0.031*"f__cnt_C1_10856" + 0.031*"f__cnt_C11_10769" + 0.031*"f__cnt_C2_10808" + 0.019*"f__cnt_C13_10929" + 0.013*"f__cnt_V265_2938" + 0.013*"f__cnt_V307_3613" + 0.013*"f__cnt_V127_3474"'),
 (3,
  '0.018*"f__M2_#" + 0.018*"f__M3_#" + 0.018*"f__M1_#" + 0.018*"f__cnt_V2_455805" + 0.018*"f__cnt_V3_455805" + 0.018*"f__cnt_V4_455805" + 0.0

In [29]:
num_topics = 8
lda8 = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)

num_topics = 12
lda12 = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)

num_topics = 16
lda16 = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)

num_topics = 20
lda20 = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)

In [129]:
prefix = '608'
trn_len = len(train_texts)
for col_no in tqdm(range(20)):
    train_feature = tmp_mx[:trn_len, col_no]
    test_feature = tmp_mx[trn_len:, col_no]
    to_pkl_gzip(path=f'../feature/create/{prefix}__lda_topic20_no{col_no}_train', obj=train_feature)
    to_pkl_gzip(path=f'../feature/create/{prefix}__lda_topic20_no{col_no}_test', obj=test_feature)



  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:09<03:00,  9.52s/it]

 10%|█         | 2/20 [00:17<02:45,  9.20s/it]

 15%|█▌        | 3/20 [00:25<02:30,  8.84s/it]

 20%|██        | 4/20 [00:34<02:19,  8.71s/it]

 25%|██▌       | 5/20 [00:38<01:51,  7.42s/it]

 30%|███       | 6/20 [00:43<01:31,  6.54s/it]

 35%|███▌      | 7/20 [00:47<01:16,  5.90s/it]

 40%|████      | 8/20 [00:52<01:05,  5.46s/it]

 45%|████▌     | 9/20 [00:56<00:56,  5.16s/it]

 50%|█████     | 10/20 [01:03<00:55,  5.58s/it]

 55%|█████▌    | 11/20 [01:12<01:01,  6.80s/it]

 60%|██████    | 12/20 [01:22<01:01,  7.64s/it]

 65%|██████▌   | 13/20 [01:31<00:57,  8.21s/it]

 70%|███████   | 14/20 [01:40<00:50,  8.41s/it]

 75%|███████▌  | 15/20 [01:46<00:38,  7.69s/it]

 80%|████████  | 16/20 [01:52<00:28,  7.22s/it]

 85%|████████▌ | 17/20 [01:59<00:21,  7.12s/it]

 90%|█████████ | 18/20 [02:09<00:15,  7.85s/it]

 95%|█████████▌| 19/20 [02:18<00:08,  8.38s/it]

100%|██████████| 20/20 [02:27<00:00,

In [133]:
def get_lda_topic(topic_num, model, corpus):
    tmp_mx = np.zeros((len(corpus), topic_num))
    for idx, sample in tqdm(enumerate(corpus)):
        topic = model[sample]
        for t_no, val in topic:
            tmp_mx[idx, t_no] = val
        
    return tmp_mx
    
topic_num = 5
topic5_mx = get_lda_topic(topic_num, lda, corpus)

prefix = '608'
trn_len = len(train_texts)
for col_no in tqdm(range(topic_num)):
    train_feature = topic5_mx[:trn_len, col_no]
    test_feature = topic5_mx[trn_len:, col_no]
    to_pkl_gzip(path=f'../feature/create/{prefix}__lda_topic{topic_num}_no{col_no}_train', obj=train_feature)
    to_pkl_gzip(path=f'../feature/create/{prefix}__lda_topic{topic_num}_no{col_no}_test', obj=test_feature)



  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:04<00:18,  4.51s/it]

 40%|████      | 2/5 [00:08<00:13,  4.49s/it]

 60%|██████    | 3/5 [00:13<00:08,  4.48s/it]

 80%|████████  | 4/5 [00:18<00:04,  4.71s/it]

100%|██████████| 5/5 [00:28<00:00,  6.23s/it]



In [ ]:
topic_num = 8
topic_mx = get_lda_topic(topic_num, lda8, corpus)

prefix = '608'
trn_len = len(train_texts)
for col_no in tqdm(range(topic_num)):
    train_feature = topic_mx[:trn_len, col_no]
    test_feature = topic_mx[trn_len:, col_no]
    to_pkl_gzip(path=f'../feature/create/{prefix}__lda_topic{topic_num}_no{col_no}_train', obj=train_feature)
    to_pkl_gzip(path=f'../feature/create/{prefix}__lda_topic{topic_num}_no{col_no}_test', obj=test_feature)
    
topic_num = 12
topic_mx = get_lda_topic(topic_num, lda12, corpus)

prefix = '608'
trn_len = len(train_texts)
for col_no in tqdm(range(topic_num)):
    train_feature = topic_mx[:trn_len, col_no]
    test_feature = topic_mx[trn_len:, col_no]
    to_pkl_gzip(path=f'../feature/create/{prefix}__lda_topic{topic_num}_no{col_no}_train', obj=train_feature)
    to_pkl_gzip(path=f'../feature/create/{prefix}__lda_topic{topic_num}_no{col_no}_test', obj=test_feature)
    
    
topic_num = 16
topic_mx = get_lda_topic(topic_num, lda16, corpus)

prefix = '608'
trn_len = len(train_texts)
for col_no in tqdm(range(topic_num)):
    train_feature = topic_mx[:trn_len, col_no]
    test_feature = topic_mx[trn_len:, col_no]
    to_pkl_gzip(path=f'../feature/create/{prefix}__lda_topic{topic_num}_no{col_no}_train', obj=train_feature)
    to_pkl_gzip(path=f'../feature/create/{prefix}__lda_topic{topic_num}_no{col_no}_test', obj=test_feature)
    
del topic_mx